In [1]:
import tensorflow as tf

print(tf.__version__)

from tensorflow.python.client import device_lib


1.15.0


In [2]:
# Mask R-CNN
import sys
import numpy as np
import skimage
import time
import os

ROOT_DIR = os.path.abspath("./")

sys.path.append(os.path.join(ROOT_DIR, "mrcnn"))
from mrcnn import utils, visualize
from imutils.video import WebcamVideoStream
import mrcnn.model as modellib
from mrcnn.model import mold_image

# Root directory of the project
from samples.coco.coco import CocoConfig



# To find local version# Directory to save logs and trained model
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))

MODEL_DIR = os.path.join(ROOT_DIR, "logs")# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "models", "mask_rcnn_coco.h5")

Using TensorFlow backend.


In [3]:
ANNOTATION_FILE = os.path.join(ROOT_DIR, "datasets", "labels_cards_in_box_COCO.json")
DATASET_IMG_DIR = os.path.join(ROOT_DIR, "datasets", "cards_in_box_resized_renamed")

import json
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw
class CardsOnBoxDataset(utils.Dataset):

  def extract_mask(self, image_id):
    with open(ANNOTATION_FILE) as f:
      labels = json.load(f)
    # masks (stored as annotations in COCO format)
    annotations = labels['annotations']
    classes = labels['categories']
    images = labels['images']
    id_coco_file = image_id
    image = next((image for image in images if image["id"] == id_coco_file))

    image_height = image['height'] #// 4
    image_width = image['width'] #// 4

    # list containing all the masks of the image of the given id/index
    image_masks = list()

    # image = next((image for image in images if image["id"] == image_id))
    image_annotations = list((annotation for annotation in annotations if annotation["image_id"] == id_coco_file))

    segmentation_tuples = list()
    class_ids = list()

    # only used for drawing
    empty_mask = np.zeros((image_height,image_width))
    # actual list of masks
    concatenated_mask = np.zeros((image_height,image_width, len(image_annotations)))

    # for each mask (annotation) belonging to the image we draw it and stack them in masks
    for i, annotation in enumerate(image_annotations):

      # mask_class = next((mask_class['name'] for mask_class in classes if mask_class["id"] == image_annotations[0]['category_id']))

      # orignal list contains a succession of x and y coordinates
      # we reshape that to obtain a list of x,y tuples
      class_ids.append(annotation['category_id'])
      segmentation_tuples = np.reshape(np.array(annotation["segmentation"][0]), (int(np.array(annotation["segmentation"][0]).size/2), 2))
      img_to_draw_on = Image.fromarray(empty_mask)
      draw = ImageDraw.Draw(img_to_draw_on)
      draw.polygon([tuple(p) for p in segmentation_tuples], fill=1)
      mask = np.asarray(img_to_draw_on)

      concatenated_mask[:, :, i] = mask

    return concatenated_mask.astype('bool_'), image_width, image_height, np.asarray(class_ids, dtype=np.int32)

  def load_cards_on_box_dataset(self):
    # Add classes
    self.add_class("cards_on_box_dataset", 1, "green_card")
    self.add_class("cards_on_box_dataset", 2, "gold_card")

    for filename in os.listdir(DATASET_IMG_DIR):

      image_id = filename[:-4]

      # if is_train and int(image_id) in [ 11,  15, 188, 187, 319, 322,  35,  67,  81,  89, 127, 234, 267, 232,  84, 88,
      #                                     3,   7, 257, 275, 222, 194, 115, 100, 230, 112, 142, 259, 226, 223, 222, 258,
      #                                   244, 272, 159, 120,   9,   2, 137,  28,  19,  23, 204, 310, 312, 303, 237, 190, ]:
      #   continue

      # if not is_train and int(image_id) not in [ 11,  15, 188, 187, 319, 322,  35,  67,  81,  89, 127, 234, 267, 232,  84, 88,
      #                                             3,   7, 257, 275, 222, 194, 115, 100, 230, 112, 142, 259, 226, 223, 222, 258,
      #                                           244, 272, 159, 120,   9,   2, 137,  28,  19,  23, 204, 310, 312, 303, 237, 190, ]:
      #   continue

      img_path = os.path.join(DATASET_IMG_DIR, filename)


      self.add_image('cards_on_box_dataset', image_id=image_id, path=img_path, annotation=ANNOTATION_FILE)



  def image_reference(self, image_id):
    info = self.image_info[image_id]
    return info['path']


  def load_mask(self, image_id):
    info = self.image_info[image_id]

    mask , w, h, class_ids= self.extract_mask(int(info['id']))

    return mask, class_ids

  def load_image(self, image_id):
    image = skimage.io.imread(self.image_info[image_id]['path'])

    return image

cards_on_box_dataset = CardsOnBoxDataset()
cards_on_box_dataset.load_cards_on_box_dataset()
cards_on_box_dataset.prepare()
print('Cards on bo dataset size: %d' % len(cards_on_box_dataset.image_ids))

Cards on bo dataset size: 85


In [4]:
class InferenceConfig(CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()# Create model object in inference mode.


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [5]:
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)# Define COCO Class names






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








In [7]:
# Time the detection
nb_images = 85
images = []
for image_id in range(nb_images):
  image_raw = cards_on_box_dataset.load_image(image_id)
  image = mold_image(image_raw, config)
  images.append(image)

print("Image loading done")

start_time = time.time()
for i in range(nb_images):
  _ = model.detect([images[i]], verbose=0)
end_time = time.time()

print(f'Time elapsed: {round(end_time - start_time, 2)} second(s)')
print(f'Mean detection time: {round((end_time - start_time)/nb_images, 2)} second')



Image loading done
Time elapsed: 122.53 second(s)
Mean detection time: 1.44 second


In [5]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
 'kite', 'baseball bat', 'baseball glove', 'skateboard',
 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
 'teddy bear', 'hair drier', 'toothbrush']

In [6]:
# Open webcam at the ID 0
cap = cv2.VideoCapture(0)#Check whether user selected camera is opened successfully.
if not (cap.isOpened()):
    print("Could not open video device")

In [ ]:
# Real-Time Webcam Object Detection
colors = visualize.random_colors(len(class_names))
gentle_grey = (45, 65, 79)
white = (255, 255, 255)
OPTIMIZE_CAM = False
SHOW_FPS = False
SHOW_FPS_WO_COUNTER = True # faster
PROCESS_IMG = True
fps_caption = "Undefined"

if OPTIMIZE_CAM:\
    vs = WebcamVideoStream(src=0).start()
else:
    vs = cv2.VideoCapture(0)


if SHOW_FPS:
    fps_caption = "FPS: 0"
    fps_counter = 0
    start_time = time.time()

SCREEN_NAME = "Real-Time Webcam"
cv2.namedWindow(SCREEN_NAME, cv2.WINDOW_NORMAL)
cv2.setWindowProperty(SCREEN_NAME, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
# Capture frame-by-frame
while True:
    if OPTIMIZE_CAM:
        frame = vs.read()
    else:
        grabbed, frame = vs.read()
    if not grabbed:
        break

    if SHOW_FPS_WO_COUNTER:
        start_time = time.time() # start time of the loop

    if PROCESS_IMG:
        results = model.detect([frame])
        r = results[0]# Run detection

        masked_image = visualize.display_instances(frame, r['rois'], r['masks'],
                                                     r['class_ids'], class_names,
                                                     r['scores'], colors=colors)

        # masked_image = visualize.display_instances_10fps(frame, r['rois'], r['masks'],
        #                                              r['class_ids'], class_names,
        #                                              r['scores'], colors=colors,
        #                                              real_time=True)


    if PROCESS_IMG:
        s = masked_image
    else:
        s = frame
    # print("Image shape: {1}x{0}".format(s.shape[0], s.shape[1]))
    #

    width = frame.shape[1]
    height = frame.shape[0]
    top_left_corner = (width-120, height-20)
    bott_right_corner = (width, height)
    top_left_corner_cvtext = (width-80, height-5)

    if SHOW_FPS:
        fps_counter+=1
        if (time.time() - start_time) > 5 : # every 5 second
            print('')
            # I commented that
            # fps_caption = "FPS: {:.0f}".format(fps_counter / (time.time() - start_time))
            # print(fps_caption)

    fps_counter = 0
    start_time = time.time()
    ret, baseline = cv2.getTextSize(fps_caption, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(s, (width - ret[0], height - ret[1] - baseline),
                  bott_right_corner, gentle_grey, -1)
    cv2.putText(s,fps_caption, (width - ret[0], height - baseline),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, lineType=cv2.LINE_AA)

    if SHOW_FPS_WO_COUNTER:
        # Display the resulting frame
        # fps_caption = "FPS: {:.0f}".format(1.0 / (time.time() - start_time))
        # print("FPS: ", 1.0 / (time.time() — start_time))
        # Put the rectangle and text on the bottom left corner
        ret, baseline = cv2.getTextSize(fps_caption, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(s, (width - ret[0], height - ret[1] - baseline),
                      bott_right_corner, gentle_grey, -1)
        cv2.putText(s, fps_caption, (width - ret[0], height - baseline),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, white, 1, lineType=cv2.LINE_AA)#Display the frame

    # s = cv2.resize(s,(1920,1080))
    s = cv2.resize(s,(1920,1080))
    cv2.imshow(SCREEN_NAME, s)
    cv2.waitKey(1)
    # if cv2.waitKey(1) & 0xFF == ord(‘q’):
    # break

# When everything is done, release the camera from video capture
if OPTIMIZE_CAM:
    vs.stop()
else:
    vs.release()
cv2.destroyAllWindows()

A:\documents\__PERSO\Pycharm_projects\MaskRCNN_realtime_webcam_detection\mrcnn\visualize.py:110: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  _, ax = plt.subplots(1, figsize=figsize)
